# Working Notebook for creating wndchrm features

## Step 1
### Load nessecary modules

In [1]:
# Add python modules

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from copy import deepcopy

# Add parent directory for custom modules
from sys import path as sysPath
sysPath.append('../')
sysPath.append('../Machine_Score/')

# Load custom modules that simr has loaded
import main_SIMR as simr
gm = simr.gm
im = simr.im
ic = simr.ic
fe = simr.fe
ms = simr.ms
sa = simr.sa
dc = simr.ms.dc

simr.test()
gm.test()
im.test()
ic.test()
fe.test()
ms.test()
dc.test()
sa.test()

SIMR: Hi!  You're in Matthew's main program for all things galaxy collisions
GM: Hi!  You're in Matthew's module for generally useful functions and classes
IM: Hi!  You're in Matthew's information module for SPAM
IC: Hi!  You're in Matthew's main code for all things image creation.
FE: Hi!  You're in Matthew's module for extracting feature values from images.
MS: Hi!  You're in Matthew's SIMR module for all things machine scoring images
DC: Hi!  You're in direct_image_compare.py
SA: Hi!  You're in Matthew's Main program for score analysis!


___
## Step 2
### Load Target Info class
Needed for loading the target image and running through models later.

In [2]:
tDir = '../targetDir'
tDir = gm.validPath(tDir)
print('tDir: %s' % tDir)
tInfo = im.target_info_class( targetDir = tDir, printAll=True)
if tInfo.status == False:
    print("WARNING: target info class bad")
else:
    print("Target Good!: %s" % tInfo.get('target_id'))

# Get run info class
rInfo = tInfo.getRunInfo( rID='run_0000' )

if rInfo.status == False:
    print("WARNING")
else:
    print("Run Good!: %s"%rInfo.get('run_id'))

tDir: /home/mbo2d/587722984435351614/
IM: target_info_class.__init__:
	 - targetDir:  /home/mbo2d/587722984435351614/
	 - arg.targetDir:  None
IM: Target.initTargetDir():
	 - targetDir: /home/mbo2d/587722984435351614/
	 - targetDir: (True) /home/mbo2d/587722984435351614/
	 - infoDir: (True) /home/mbo2d/587722984435351614/information/
	 - baseInfoLoc: (True) /home/mbo2d/587722984435351614/information/base_target_info.json
	 - allInfoLoc: (True) /home/mbo2d/587722984435351614/information/target_info.json
	 - zooMergerDir: (True) /home/mbo2d/587722984435351614/zoo_merger_models/
	 - plotDir: (True) /home/mbo2d/587722984435351614/plots/
IM: Target: Opening target info json
Target Good!: 587722984435351614
Run Good!: run_0000


In [3]:
chime_0 = tInfo.readScoreParam( 'chime_0')
chime_group_1 = tInfo.readScoreParam( 'chime_group_1')
norm_chime_1 = tInfo.readWndchrmNorm( 'norm_chime_1')
print(norm_chime_1)

{'name': 'norm_chime_1', 'image_group': 'chime_group_1', 'normalization_method': 'sklearn_StandardScaler', 'top_models': 500}


In [4]:
testArg = gm.inArgClass()
#testArg.setArg('newFeats',True)
#testArg.setArg('newImage',True)
testArg.setArg('printAll',True)
testArg.setArg( 'scoreParams', chime_group_1 )
testArg.setArg( 'normName', 'norm_chime_1' )

#testArg.printArg()

#simr.simr_run( arg=testArg, rInfo=rInfo )

In [5]:

# Exploring wndchrm_all features
def analyze_wndchrm_dataframe( args, tInfo, ):
    
    # Grab useful variables
    printAll = args.printAll
    printBase = args.printBase
    normName = args.get('normName', None)
    featDF = None
    
    infoHeaders = tInfo.wndchrmInfoHeaders    
    
    if printBase:    print("FE: analyze_wndchrm_dataframe")
    
    # Analysis requires a normalized dataset of the wndchrm features
    # Use normalize info to read the right dataframe
    if normName == None:
        if printBase: 
            print("WARNING: FE: analyze_wndchrm_dataframe")
            gm.tabprint("Please provide `-normName file_name`")
        return
    
    elif type(normName) == type('string'):
        
        featDF = tInfo.readWndchrmDF( normName )
        
        # If dataframe read
        if type(featDF) == type(pd.DataFrame()):
            if printBase:
                gm.tabprint("Read dataframe: %s" % normName)
                gm.tabprint("Dataframe Shape: %s" % str(featDF.shape))
                
        # Didn't read a dataframe
        else:
            if printBase:
                print( "WARNING: FE: analyze_wndchrm_dataframe" )
                gm.tabprint( "Unable to read dataframe: %s" % normName)
                gm.tabprint( "Dataframe: %s" % type(featDF) )
            return
    
    # Seperate into info, scores, and features
    infoDF = featDF.filter( infoHeaders, axis=1 )
    feats = featDF.drop( infoHeaders, axis=1 ).values
    scores = infoDF['zoo_merger_score'].values

    # Variance analysis
    fe.variance_analysis( feats, args, tInfo )    
    
#analyze_wndchrm_dataframe( testArg, tInfo )

In [6]:
      

df1 = fe.target_apply_filter( testArg, tInfo, )
print(df1.shape)

FE: target_apply_filter:
	 - Gathering images names to filter
	 - Found 48 unique images
	 - Reading Normalized WNCHRM Features: norm_chime_1
	 - 	Starting Shape: (63213, 1063)
	 - Filtering scores by image
	 - 	Filtered shape: (61921, 1063)
(61921, 1063)


In [7]:
def feat_select_mutual_info( args = None, \
                            df = None, 
                            ignore_headers = ['image_name', 'target_id', 'run_id'], \
                            y_header = 'zoo_merger_score'):
    
    from sklearn.feature_selection import mutual_info_regression
    
    if args == None:  args = gm.inArgClass()
    printBase = args.printBase
    printAll = args.printAll
    
    if printBase: print("FE: feat_select_mutual_info:")
        
    if type(df) == type(None):
        if printBase: print("WARNING: FE: feat_select_mutual_info: Please provide dataframe")
        return
    
    if printAll:
        gm.tabprint("Starting DF shape: %s" % str( df.shape) )
        
    # Filter bad values/scores
    df = df.drop( columns=ignore_headers )
    
    filtDF = df[ ~df.isin( [np.nan, np.inf, -np.inf]).any(1) ]
    
    if printAll:
        gm.tabprint("Filtered DF shape: %s" % str( filtDF.shape) )
    
    y = filtDF[y_header].values    
    X = filtDF.drop( columns=y_header ).values
    
    if printAll:
        gm.tabprint("X shape: %s" % str( X.shape) )
        gm.tabprint("Y shape: %s" % str( y.shape ) )
        gm.tabprint("Starting mutual info regression")
    
    X_reg = mutual_info_regression(X, y, n_neighbors=3, )
    
    if printAll:
        gm.tabprint("Mutual Info Regression Complete")
        
    if printBase: print("FE: feat_select_mutual_info: Complete")
        
    return X, y, X_reg
    
X, y, X_reg = feat_select_mutual_info( testArg, df1 )

FE: feat_select_mutual_info:
	 - Starting DF shape: (61921, 1063)
	 - Filtered DF shape: (61910, 1060)
	 - X shape: (61910, 1059)
	 - Y shape: (61910,)
	 - Starting mutual info regression
	 - Mutual Info Regression Complete
FE: feat_select_mutual_info: Complete


In [8]:
print(X_reg)
print(X_reg.shape)

wndchrm_headers = df1.columns[4:]
print(wndchrm_headers.shape)



[0.0918784  0.05879705 0.09965687 ... 0.12180646 0.07249799 0.09074317]
(1059,)
(1059,)


In [9]:

n = 10
topClassInd = X_reg.argsort()[-n:][::-1]
topClass = X_reg[topClassInd]

for i in range(n):
    print('%4d: %.3f   %s'%(topClassInd[i],topClass[i],cList[topClassInd[i]]))

NameError: name 'cList' is not defined